# Load libraries

In [22]:
from pygments import highlight
from pygments.lexers import PythonLexer
from pygments.formatters import Terminal256Formatter
from pprint import pformat

import tensorflow as tf
import numpy as np
from pprint import pprint
print(tf.__version__)

def pprint_color(obj):
    print(highlight(pformat(obj), PythonLexer(), Terminal256Formatter()))

1.12.0


# Intro

In [23]:
# 문장의 단어를 RNN에 하나 하나씩 넣는다고 하면?
sentences = [['I', 'feel', 'hungry'],
            ['tensorflow', 'is', 'verry', 'difficult'],
            ['tensorflow', 'is', 'a', 'framework', 'for', 'deep', 'learning'],
            ['tensorflow', 'is', 'very', 'fast', 'change']]

# RNN은 아래처럼 각 문장 별로 단어의 개수만큼 sequence를 처리해야한다.
# --> variable sequence lenghth!
print(list(map(lambda word : len(word), sentences)))

[3, 4, 7, 5]


## Intro : Padding

In [35]:
# word dic
word_list = []
pprint_color("Merging : 하나의 리스트로 표현")
pprint_color("Before")
print(sentences, end="\n\n")

for elm in sentences:
    word_list += elm

pprint_color("After")
print(word_list, end='\n\n\n\n')
    
    
pprint_color("What is mean that set(word_list) : 중복제거")
pprint_color("Before")
print(word_list, end='\n\n')
pprint_color("After")
print(set(word_list), end='\n\n\n\n')

# set으로 만들어주는 이유는 중복 제거
pprint_color("make list after set(word_list) : 중복 제거 후, 리스트로 재변환")
word_list = list(set(word_list))
print(word_list, end='\n\n\n\n')

# string 리스트에서 sort는 어떤 의미를 갖는가?
# https://thispointer.com/python-how-to-sort-a-list-of-strings-list-sort-tutorial-examples/
# sort은 낮은 순서에서 높은 순서로 정렬
# 알파벳 순서
word_list.sort()
pprint_color("Sorting : 알파벳순으로 정렬")
print(word_list, end='\n\n\n\n')

# token은 왜?
pprint_color("padding : 왜 <pad> token을 붙이나요 보섭선생님?")
word_list = ['<pad>'] + word_list # '<pad>'라는 의미없는 token 추가
print(word_list, end='\n\n\n\n')

# string마다 idx로 정수화
pprint_color("string마다 idx붙여서 정수화?")
word_dic = {word :idx for idx, word in enumerate(word_list)}
pprint(word_dic)

'Merging : 하나의 리스트로 표현'

'Before'

[['I', 'feel', 'hungry'], ['tensorflow', 'is', 'verry', 'difficult'], ['tensorflow', 'is', 'a', 'framework', 'for', 'deep', 'learning'], ['tensorflow', 'is', 'very', 'fast', 'change']]

'After'

['I', 'feel', 'hungry', 'tensorflow', 'is', 'verry', 'difficult', 'tensorflow', 'is', 'a', 'framework', 'for', 'deep', 'learning', 'tensorflow', 'is', 'very', 'fast', 'change']



'What is mean that set(word_list) : 중복제거'

'Before'

['I', 'feel', 'hungry', 'tensorflow', 'is', 'verry', 'difficult', 'tensorflow', 'is', 'a', 'framework', 'for', 'deep', 'learning', 'tensorflow', 'is', 'very', 'fast', 'change']

'After'

{'for', 'hungry', 'tensorflow', 'difficult', 'framework', 'I', 'is', 'a', 'very', 'feel', 'change', 'verry', 'deep', 'fast', 'learning'}



'make list after set(word_list) : 중복 제거 후, 리스트로 재변환'

['for', 'hungry', 'tensorflow', 'difficult', 'framework', 'I', 'is', 'a', 'very', 'feel', 'change', 'verry', 'deep', 'fast', 'learning']



'Sorting : 알파벳순으

In [61]:
# max_len의 길이에 못미치는 문장은 <pad>로 max_len만큼 padding
def pad_seq(sequences, max_len, dic):
    seq_len, seq_indices = [], []
    for seq in sequences:
        # 길이를 list에 appendix
        # 즉, 해당 원소의 유의미한 elements 개수를 확인
        seq_len.append(len(seq))
        
        # 문장의 단어를 만들어놓은 idx로 정수화
        seq_idx = [dic.get(char) for char in seq]
        
        # dic.get('<pad>') => '<pad>'의 idx값을 뽑아옴
        # len(seq_idx) => 문장의 길이를 구함 
        # 즉, 고정된 최대 길이에서 얼만큼의 length가 여유가 있는지 확인
        # 그 이후, 남는 길이만큼 기존 seq_idx 뒤에 `<pad>` idx값을 padding해줌.
        # seq_indices는 text를 정수화하고 정수화 된 배열을 모아두는 곳.
        # 일종의 input data 느낌
        seq_idx += (max_len - len(seq_idx)) * [dic.get('<pad>')] # 0 is idx of meaningless token "<pad>"
        seq_indices.append(seq_idx)    
    return seq_len, seq_indices

In [72]:
max_length = 8
sen_len, sen_indices = pad_seq(sequences = sentences, max_len = max_length, dic = word_dic)
pprint(sen_len)
pprint(sen_indices)
print("\n\n")

sen_indices = np.asarray(sen_indices)
pprint_color("list data type cast to numpy array")
print(sen_indices)
print(sen_indices.shape)

[3, 4, 7, 5]
[[1, 7, 10, 0, 0, 0, 0, 0],
 [13, 11, 14, 5, 0, 0, 0, 0],
 [13, 11, 2, 9, 8, 4, 12, 0],
 [13, 11, 15, 6, 3, 0, 0, 0]]



'list data type cast to numpy array'

[[ 1  7 10  0  0  0  0  0]
 [13 11 14  5  0  0  0  0]
 [13 11  2  9  8  4 12  0]
 [13 11 15  6  3  0  0  0]]
(4, 8)


## Intro : Padding

In [63]:
# 위에서 했던 작업을 tf로 하는 내용
seq_len = tf.placeholder(dtype = tf.int32, shape = [None])
seq_indices = tf.placeholder(dtype = tf.int32, shape = [None, max_length])

In [64]:
# 이미 padding되어있는 빈 tensor를 생성
one_hot = np.eye(len(word_dic)).astype(np.float32)
# 초기값을 paddingg값으로 채워진 빈 tensor로 'one_hot'이라는 변수를 생성
# placeholder가 아니라 variable로 하는 이유는? => tf.nn.embedding_lookup의 params Arg가 tf.get_variable()
one_hot = tf.get_variable(name='one_hot', initializer = one_hot,
                         trainable = False)

#  params may be a PartitionedVariable as returned by using tf.get_variable() with a partitioner.
seq_batch = tf.nn.embedding_lookup(params = one_hot, ids = seq_indices)

In [75]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    tmp = sess.run(seq_batch, feed_dict = {seq_indices : sen_indices})
    
# 위에서는 shape이 (4,8)이었으나
# one-hot encoding되기 때문에 idx가 0~15인점을 감안해서 shape은 (4, 8, 16)
print(np.shape(sen_indices))
print(np.shape(tmp))
print(tmp)

(4, 8)
(4, 8, 16)
[[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 